# Tune convolution filters




In [ ]:
import numpy as np
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier


import tensorflow as tf
import keras

num_classes = 25

In [ ]:
def create_model(filter=32):
  input = tf.keras.Input(shape=(64,64,3))
  # Block 1
  x = tf.keras.layers.Conv2D(filter, kernel_size=6, activation='relu')(input)
  x = tf.keras.layers.MaxPooling2D((5,5))(x)

  x = tf.keras.layers.Dropout(.5)(x)
  x = tf.keras.layers.Flatten()(x)

  x = tf.keras.layers.Dense(50, activation='relu')(x)
  output = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

  # bind all
  model = tf.keras.Model(input, output)

  model.compile(
    loss = keras.losses.CategoricalCrossentropy(),
    optimizer = keras.optimizers.Adam(),
    metrics=["accuracy",tf.keras.metrics.CategoricalAccuracy()]
  )
  return model

In [ ]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

In [ ]:
X = np.load('/content/drive/MyDrive/Colab Notebooks/cnn_img_avril/imgs.npy')
Y = np.load('/content/drive/MyDrive/Colab Notebooks/cnn_img_avril/labels.npy')

In [ ]:
# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters
filters = [10, 24, 32, 64]
param_grid = dict(filter=filters)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Best: 0.434518 using {'filter': 64}
0.336535 (0.008855) with: {'filter': 10}
0.332586 (0.003933) with: {'filter': 24}
0.420429 (0.126352) with: {'filter': 32}
0.434518 (0.190952) with: {'filter': 64}
